# Tool recommendation with Attention network 
## (Gated recurrent units Attention neural network with weighted cross-entropy loss)

In [66]:
import numpy as np
import json
import warnings
import operator

import tensorflow as tf
from keras import backend as K
import h5py

warnings.filterwarnings("ignore")


class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        self.units = units
        super(BahdanauAttention, self).__init__(**kwargs)

    def get_config(self):
        return super().get_config().copy()

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights


class ToolPredictionAttentionModel():
  
    def __init__(self, parameters):
        self.embedding_size = int(parameters["embedding_size"])
        self.gru_units = int(parameters["units"])
        self.max_len = parameters["max_len"]
        self.dimensions = parameters["dimensions"]
        self.learning_rate = parameters["learning_rate"]
        self.class_weights = parameters["class_weights"]
        
    def weighted_loss(self, class_weights):
        """
        Create a weighted loss function. Penalise the misclassification
        of classes more with the higher usage
        """
        weight_values = list(class_weights.values())

        def weighted_binary_crossentropy(y_true, y_pred):
            # add another dimension to compute dot product
            expanded_weights = tf.keras.backend.expand_dims(weight_values, axis=-1)
            return tf.keras.backend.dot(tf.keras.backend.binary_crossentropy(y_true, y_pred), expanded_weights)
        return weighted_binary_crossentropy

    def create_model(self):
        sequence_input = tf.keras.layers.Input(shape=(self.max_len,), dtype='int32')
        embedded_sequences = tf.keras.layers.Embedding(self.dimensions, self.embedding_size, input_length=self.max_len, mask_zero=True)(sequence_input)
        gru = tf.keras.layers.GRU(self.gru_units,
            return_sequences=True,
            return_state=True,
            activation='elu'
        )
        sample_output, sample_hidden = gru(embedded_sequences, initial_state=None)
        attention = BahdanauAttention(self.gru_units)
        context_vector, attention_weights = attention(sample_hidden, sample_output)
        output = tf.keras.layers.Dense(self.dimensions, activation='sigmoid')(context_vector)
        model = tf.keras.Model(inputs=sequence_input, outputs=output)
        model.compile(
            optimizer=tf.keras.optimizers.RMSprop(learning_rate=self.learning_rate),
            loss=self.weighted_loss(self.class_weights),
        )
        return model


def load_model(model_path):
    trained_model = h5py.File(model_path, 'r')
    dictionary = json.loads(trained_model.get('data_dictionary').value)
    best_parameters = json.loads(trained_model.get('parameters').value)
    compatible_tools = json.loads(trained_model.get('compatible_tools').value)
    reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
    new_model = ToolPredictionAttentionModel(best_parameters).create_model()
    new_model.load_weights(model_path)
    return dictionary, reverse_dictionary, class_weights, best_parameters, compatible_tools, new_model


model_path = "data/tool_recommendation_attention_model.hdf5"
dictionary, reverse_dictionary, class_weights, best_parameters, compatible_tools, new_model = load_model(model_path)

## Unpack trained model for prediction

In [67]:
print(reverse_dictionary)

{'1': 'cshl_sort_header', '2': 'CONVERTER_bed_gff_or_vcf_to_bigwig_0', '3': 'meme_meme', '4': 'rmcontamination', '5': 'EMBOSS: water107', '6': 'ctb_pubchem_download_as_smiles', '7': 'interproscan', '8': 'deeptools_bamFingerprint', '9': 'macs2_bdgcmp', '10': 'regex_replace', '11': 'Grouping1', '12': 'sam_bw_filter', '13': 'bedtools_intersectBed', '14': 'sam2interval', '15': 'sort1', '16': 'seq_filter_by_id', '17': 'FileMerger', '18': 'cummeRbund', '19': 'picard_ValidateSamFile', '20': 'bedtools_bamtobed', '21': 'mergeCols1', '22': 'fragmenter', '23': 'ProteinQuantifier', '24': 'modencode_peakcalling_macs2', '25': 'EMBOSS: fuzztran39', '26': 'hgv_david', '27': 'gops_intersect_1', '28': 'vcftools_isec', '29': 'tp_easyjoin_tool', '30': 'Summary_Statistics1', '31': 'tab2fasta', '32': 'CONVERTER_gff_to_bed_0', '33': 'antismash', '34': 'extract_bcs.py', '35': 'rseqc_infer_experiment', '36': 'Paste1', '37': 'aragorn_trna', '38': 'ngsutils_bam_filter', '39': 'bedtools_coveragebed', '40': 'bamCo

In [68]:
def compute_recommendations(tool_sequence, labels, topk=20, max_seq_len=25):
    tl_seq = tool_sequence.split(",")
    last_tool_name = reverse_dictionary[str(tl_seq[-1])]
    sample = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))
    tool_sequence_names = [reverse_dictionary[str(tool_pos)] for tool_pos in tool_sequence.split(",")]
    # predict next tools for a test path
    prediction = new_model.predict(sample_reshaped, verbose=0)
    class_weighs = best_parameters["class_weights"]
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    prediction = np.reshape(prediction, (prediction.shape[1],))
    #prediction = prediction * weight_val
    prediction = prediction / float(np.max(prediction))
    prediction_pos = np.argsort(prediction, axis=-1)
    # get topk prediction
    topk_prediction_pos = prediction_pos[-topk:]
    topk_prediction_pos = [item for item in topk_prediction_pos if item != 0]
    topk_prediction_val = [int(prediction[pos] * 100) for pos in topk_prediction_pos]
    # read tool names using reverse dictionary
    pred_tool_ids = [reverse_dictionary[str(tool_pos)] for tool_pos in topk_prediction_pos]
    pred_tool_ids_sorted = dict()
    for (tool_pos, tool_pred_val) in zip(topk_prediction_pos, topk_prediction_val):
        tool_name = reverse_dictionary[str(tool_pos)]
        pred_tool_ids_sorted[tool_name] = tool_pred_val
    pred_tool_ids_sorted = dict(sorted(pred_tool_ids_sorted.items(), key=lambda kv: kv[1], reverse=True))
    ids_tools = dict()
    keys = list(pred_tool_ids_sorted.keys())
    tool_seq_name = ",".join(tool_sequence_names)
    print("Current tool sequence: ")
    print()
    print(tool_seq_name)
    print()
    print("Recommended tools for the tool sequence '%s' with their scores in decreasing order:" % tool_seq_name)
    print()
    for i in pred_tool_ids_sorted:
        print(i + "(" + str(pred_tool_ids_sorted[i]) + "%)")
    for key in pred_tool_ids_sorted:
        ids_tools[key] = dictionary[key]
    print()
    print("Tool ids:")
    print()
    for i in ids_tools:
        print(i + "(" + str(ids_tools[i]) + ")")

## Indices of tools

In [69]:
topk = 20 # set the maximum number of recommendations
tool_seq = "2" # give tools ids in a sequence and see the recommendations. To know all the tool ids, 
                     # please print the variable 'reverse_dictionary'
compute_recommendations(tool_seq, "", topk)

Current tool sequence: 

CONVERTER_bed_gff_or_vcf_to_bigwig_0

Recommended tools for the tool sequence 'CONVERTER_bed_gff_or_vcf_to_bigwig_0' with their scores in decreasing order:

get_flanks1(100%)
ctb_filter(99%)
modencode_peakcalling_macs2(97%)
bam_to_sam(96%)
deeptools_heatmapper(95%)
rmcontamination(95%)
bedtools_intersectBed(94%)
Show tail1(93%)
flexbar_split_RYYR_bcs(93%)
gatk2_variant_select(92%)
flexbardsc(92%)
r_correlation_matrix(92%)
DatamashTranspose(91%)
deeptools_bamFingerprint(91%)
eukaryotic_ncbi_submission(91%)
tp_awk_tool(91%)
term_id_vs_term_def(91%)
computeMatrix(90%)
silac_analyzer(90%)
cshl_multijoin(90%)

Tool ids:

get_flanks1(243)
ctb_filter(109)
modencode_peakcalling_macs2(24)
bam_to_sam(168)
deeptools_heatmapper(149)
rmcontamination(4)
bedtools_intersectBed(13)
Show tail1(284)
flexbar_split_RYYR_bcs(73)
gatk2_variant_select(52)
flexbardsc(46)
r_correlation_matrix(248)
DatamashTranspose(162)
deeptools_bamFingerprint(8)
eukaryotic_ncbi_submission(247)
tp_awk_

## Recommended tools

In [59]:
print(new_model.summary())

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 25, 119)           36890     
_________________________________________________________________
gru_8 (GRU)                  [(None, 25, 97), (None, 9 63438     
_________________________________________________________________
bahdanau_attention_7 (Bahdan ((None, 97), (None, 25, 1 19110     
_________________________________________________________________
dense_31 (Dense)             (None, 310)               30380     
Total params: 149,818
Trainable params: 149,818
Non-trainable params: 0
_________________________________________________________________
None
